In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
from scipy import stats
from scipy.stats import norm, skew

In [26]:
import seaborn as sb

In [27]:
#EDA
#possible Missing values for columns
miss_val=['NaN']

In [28]:
# Reading Data

In [29]:
aisles=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/aisles.csv', na_values=miss_val)

In [30]:
departments=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/departments.csv', na_values=miss_val)

In [31]:
products=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/products.csv', na_values=miss_val)
#products.isnull().values.any()
#products.head(10)

In [32]:
order_products_prior=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/order_products__prior.csv', na_values=miss_val)

In [33]:
order_products_train=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/order_products__train.csv', na_values=miss_val)

In [34]:
orders=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/orders.csv', na_values=miss_val)
orders.fillna(-1, inplace=True) # Fill missing values with -1

In [35]:
# Combining dataframes orders_prior and orders_train
order_products_total=pd.concat([order_products_prior, order_products_train], axis=0)
order_products_total.shape

(33819106, 4)

In [36]:
order_products_usr=pd.merge(orders, order_products_total, on='order_id')
#order_products_usr.isnull().sum()
order_products_usr.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,-1.0,196,1,0
1,2539329,1,prior,1,2,8,-1.0,14084,2,0
2,2539329,1,prior,1,2,8,-1.0,12427,3,0
3,2539329,1,prior,1,2,8,-1.0,26088,4,0
4,2539329,1,prior,1,2,8,-1.0,26405,5,0
5,2398795,1,prior,2,3,7,15.0,196,1,1
6,2398795,1,prior,2,3,7,15.0,10258,2,0
7,2398795,1,prior,2,3,7,15.0,12427,3,1
8,2398795,1,prior,2,3,7,15.0,13176,4,0
9,2398795,1,prior,2,3,7,15.0,26088,5,1


In [37]:
#Most ordered product by all users
most_freq_prod=order_products_usr.groupby('product_id').size().sort_values(ascending=False)
print(most_freq_prod)

product_id
24852    491291
13176    394930
21137    275577
21903    251705
47209    220877
          ...  
7626          1
14485         1
43098         1
11254         1
36233         1
Length: 49685, dtype: int64


In [38]:
# Total number of times each product was bought by every user
freq_user_product=order_products_usr.groupby('user_id')['product_id'].value_counts()
    #freq_user_product[i]=order_products_usr[order_products_usr['user_id']==i]['product_id'].value_counts()
    #most_freq_product_user[i]=freq_user_product[i].index[0]
    #print("Most frequent product for User {} is {}\n\n".format(i,most_freq_product_user[i]))
print(freq_user_product)

user_id  product_id
1        196           11
         10258         10
         12427         10
         25133          9
         13032          4
                       ..
206209   41665          1
         42606          1
         44325          1
         48370          1
         48697          1
Name: product_id, Length: 13863746, dtype: int64


In [39]:
# Total number of unique orders for every user
total_orders_user=order_products_usr.groupby('user_id')['order_id'].nunique().reset_index().rename(columns={'order_id':'total_unique_orders'})
print(total_orders_user)

        user_id  total_unique_orders
0             1                   11
1             2                   15
2             3                   12
3             4                    5
4             5                    5
...         ...                  ...
206204   206205                    4
206205   206206                   67
206206   206207                   16
206207   206208                   49
206208   206209                   14

[206209 rows x 2 columns]


In [40]:
# This gives the total number of products in EACH order against all users.
# Segregation of the below values.
total_orders_user3=order_products_usr.groupby('user_id')['order_id'].value_counts() 
print(total_orders_user3)

user_id  order_id
1        1187899     11
         2550362      9
         431534       8
         2295261      6
         2398795      6
                     ..
206209   272231       8
         1854736      8
         2307371      3
         2558525      3
         3186442      2
Name: order_id, Length: 3346083, dtype: int64


In [41]:
# This gives the total number of orders for all users. Sum of values above
total_orders_user2=order_products_usr.groupby('user_id')['order_id'].count().reset_index().rename(columns={'order_id':'count_orders'})
print(total_orders_user2)

        user_id  count_orders
0             1            70
1             2           226
2             3            88
3             4            18
4             5            46
...         ...           ...
206204   206205            51
206205   206206           285
206206   206207           223
206207   206208           677
206208   206209           137

[206209 rows x 2 columns]


In [42]:
torders=0
for key, value in total_orders_user3.items():
    if (key[0]==1):
        torders+=value

print(torders)

70


In [52]:
#Number of unique products ordered by every user
unique_product_user=order_products_usr.groupby('user_id')['product_id'].nunique().reset_index().rename(columns={'product_id':'num_unique_products'})
print(unique_product_user)    

        user_id  num_unique_products
0             1                   19
1             2                  121
2             3                   33
3             4                   17
4             5                   28
...         ...                  ...
206204   206205                   37
206205   206206                  150
206206   206207                   92
206207   206208                  198
206208   206209                   72

[206209 rows x 2 columns]


In [53]:
#new_product_ratio for every user= unique products/total products
nd=pd.merge(unique_product_user, total_orders_user2, on="user_id")
#nd['new_product_ratio'] = nd.apply(lambda row: row.unique_product_count/row.total_product_count*100, axis=1)
nd['new_product_ratio'] = nd.apply(lambda row: row.num_unique_products/row.count_orders*100, axis=1)
print(nd)

        user_id  num_unique_products  count_orders  new_product_ratio
0             1                   19            70          27.142857
1             2                  121           226          53.539823
2             3                   33            88          37.500000
3             4                   17            18          94.444444
4             5                   28            46          60.869565
...         ...                  ...           ...                ...
206204   206205                   37            51          72.549020
206205   206206                  150           285          52.631579
206206   206207                   92           223          41.255605
206207   206208                  198           677          29.246677
206208   206209                   72           137          52.554745

[206209 rows x 4 columns]


In [45]:
#Product ordering ratio for every user = total number of orders containing product/ total number of orders
#How many times a user has reordered a product?
order_contains_product=order_products_usr.groupby(['user_id', 'product_id'])['order_id'].agg(['count']).reset_index().rename(columns={'count': "orders_with_product"})
total_user_order=order_products_usr.groupby('user_id')['order_id'].agg(['count']).reset_index().rename(columns={'count': "total_orders"})
nro=pd.merge(order_contains_product, total_user_order, on="user_id")
nro['product_order_ratio']=nro.apply(lambda row: row.orders_with_product/row.total_orders*100, axis=1)
print(nro)

          user_id  product_id  orders_with_product  total_orders  \
0               1         196                   11            70   
1               1       10258                   10            70   
2               1       10326                    1            70   
3               1       12427                   10            70   
4               1       13032                    4            70   
...           ...         ...                  ...           ...   
13863741   206209       43961                    3           137   
13863742   206209       44325                    1           137   
13863743   206209       48370                    1           137   
13863744   206209       48697                    1           137   
13863745   206209       48742                    2           137   

          product_order_ratio  
0                   15.714286  
1                   14.285714  
2                    1.428571  
3                   14.285714  
4                    5.

In [ ]:
print(total_user_order.iloc[0]) #iloc takes specific row number as paramter.

In [ ]:
# Frequency of products ordered by every user in each order
#user_order_products22=order_products_usr.groupby(['user_id', 'order_id'])['product_id'].count() # returns a Series
user_order_products=order_products_usr.groupby(['user_id', 'order_id'])['product_id'].agg(['count',  'mean']) # returns a Dataframe
for a in user_order_products.itertuples():
    print(a[0], a[1], a[2])

In [ ]:
# Most Frequent time for every user
user_time=order_products_usr.groupby('user_id')['order_hour_of_day'].value_counts()

In [ ]:
#Most frequent Time of purchase for the shop
most_popular_time=order_products_usr.groupby('order_hour_of_day')['user_id'].count()

In [ ]:
# Most frequent day of purchase for every user
user_day=order_products_usr.groupby('user_id')['order_dow'].value_counts()

In [ ]:
# Most frequent day of purchase for shop
most_popular_day=order_products_usr.groupby('order_dow')['user_id'].count()

In [ ]:
order_products_usr.head()

In [ ]:
departments.head(2)

In [ ]:
#Most ordered product in every department
most_popular_product_department=

In [ ]:
#Did any user reorder their previous order

In [ ]:
#Most frequent time when users purchase

In [ ]:
#Times a customer orders from the same department. Wrt to customer

In [ ]:
#Days a customer orders the same products

In [ ]:
#Times a product is ordered from the same department. Wrt product